In [24]:
# import block
import numpy as np
from astropy.io import fits
%matplotlib inline
from matplotlib import pyplot as plt
from astropy.visualization import hist
import ccdproc as ccdp
from matplotlib import rc
from datetime import datetime
from ccdproc import ImageFileCollection
import ccdproc as ccdp
from astropy.modeling.models import Polynomial1D,Chebyshev1D,Legendre1D,Hermite1D, Gaussian1D
from astropy.modeling import fitting
from astropy.stats import mad_std
from astropy.nddata import CCDData
from scipy import stats
from astropy import units as u

First, we import our convience functions

In [25]:
# import convenience plotting functions downloaded from 
# here: https://github.com/mwcraig/ccd-reduction-and-photometry-guide
phot_tutorial_dir = '/Users/azeem/Downloads/Software/py/ccd-reduction-and-photometry-guide/ccd-reduction-and-photometry-guide/notebooks/'
import sys
sys.path.insert(0,phot_tutorial_dir)
from convenience_functions import show_image

We then make insert directories for our raw data and where our reduced data will go

In [39]:
data_dir = '/Users/azeem/Downloads/Imaging/'
reduced_dir = '/Users/azeem/Downloads/Reduced/'

We now read in the bias frames while introducing key words in our headers. This will let us easily trim and grab these files. We determine that the overscan region we subtract our data from is from 2141:2200. Additionally, to trim off the overscan region, we determine the region to keep is from 54:2101 

In [27]:
# read in the biases, they're in a093-a111.fits
bias_files = ['a'+str(i).zfill(3)+'.fits' for i in np.arange(19)+93]
hdus = [fits.open(data_dir+file) for file in bias_files]
hdrs = [hdu[0].header for hdu in hdus]
    
for i in range(np.size(hdus)):
    hdrs[i]['imagetyp'] = 'bias'
    hdrs[i]['object']   = 'none'
    hdrs[i]['biassec']  = '[2141:2200,1:2048]' # FITS indexing, row/col reversed
    hdrs[i]['trimsec']  = '[54:2101,1:2048]'
    hdrs[i]['filter']   = 'none'
    
    hdus[i].writeto(reduced_dir+bias_files[i],overwrite=True)
    
del hdus
del hdrs

We do the same for the sky flats. There are saturated flats that need to be removed. Also, we should specify which flat goes with what filter.

In [28]:
# read in the flats, they're in a005-a062.fits

flat_files = ['a'+str(i).zfill(3)+'.fits' for i in np.arange(58)+5]

# delete saturated/bad flats from the list
del flat_files[16], flat_files[23:26], flat_files[47]

hdus = [fits.open(data_dir+file) for file in flat_files]
hdrs = [hdu[0].header for hdu in hdus]

for i in range(np.size(hdus)):
    hdrs[i]['imagetyp'] = 'skyflat'
    hdrs[i]['object']   = 'none'
    hdrs[i]['biassec']  = '[2141:2200,1:2048]'# FITS standard
    hdrs[i]['trimsec']  = '[54:2101,1:2048]'
    
    if ((i>=16) & (i<=22)):
        hdrs[i]['filter']   = 'U'
    elif ((i>=22) & (i<= 33)):
        hdrs[i]['filter']   = 'B'
    elif ((i>=33) & (i<= 40)):
        hdrs[i]['filter']   = 'V'        
    elif ((i>=40) & (i<= 46)):
        hdrs[i]['filter']   = 'R'        
    elif ((i>=46) & (i<= 52)):
        hdrs[i]['filter']   = 'I'  
    else:
        hdrs[i]['filter']   = 'Ha'
    
    hdus[i].writeto(reduced_dir+flat_files[i],overwrite=True)

del hdus
del hdrs

As well as the dark frames.

In [29]:
dark_files = ['d'+str(i).zfill(3)+'.fits' for i in np.arange(15)+1]
hdus = [fits.open(data_dir+file) for file in dark_files]
hdrs = [hdu[0].header for hdu in hdus]
    
for i in range(np.size(hdus)):
    hdrs[i]['imagetyp'] = 'dark'
    hdrs[i]['biassec']  = '[2141:2200,1:2048]' # FITS standard
    hdrs[i]['trimsec']  = '[54:2101,1:2048]'
    hdrs[i]['filter']   = 'none'
    
    hdus[i].writeto(reduced_dir+dark_files[i],overwrite=True)
    
del hdus
del hdrs

We want to do the same for the PG1633 science frames. However, since they are scattered in the file list we first generate their full range, then delete intervals to get what we need.

In [30]:
science_files = ['a'+str(i).zfill(3)+'.fits' for i in np.arange(173)+74]
del science_files[19:56]
del science_files[29:91]
del science_files[39:64]

hdus = [fits.open(data_dir+file) for file in science_files]
hdrs = [hdu[0].header for hdu in hdus]

for i in range(np.size(hdus)):
    hdrs[i]['imagetyp'] = 'science'
    hdrs[i]['object']   = 'pg1633'
    hdrs[i]['biassec']  = '[2141:2200,1:2048]' # FITS standard
    hdrs[i]['trimsec']  = '[54:2101,1:2048]'
    
    if ((i>=7) & (i<=8)) | ((i>=19) & (i<=20)) | ((i>=29) & (i<=30)) | ((i>=39) & (i<=40)):
        hdrs[i]['filter']   = 'U'
    elif ((i>=8) & (i<=10)) | ((i>=20) & (i<=22)) | ((i>=30) & (i<=32)) | ((i>=40) & (i<=42)):
        hdrs[i]['filter']   = 'B' 
    elif ((i>=0) & (i<=6)) | ((i>=10) & (i<=12)) | ((i>=22) & (i<=24)) | ((i>=32) & (i<=34)) | ((i>=42) & (i<=44)):
        hdrs[i]['filter']   = 'V'        
    elif ((i>=12) & (i<=14)) | ((i>=24) & (i<=26)) | ((i>=34) & (i<=36)) | ((i>=44) & (i<=46)):
        hdrs[i]['filter']   = 'R'        
    elif ((i>=14) & (i<=16)) | ((i>=26) & (i<=28)) | ((i>=36) & (i<=38)) | ((i>=46) & (i<=48)):
        hdrs[i]['filter']   = 'I'  
    else:
        hdrs[i]['filter']   = 'Ha'
    
    hdus[i].writeto(reduced_dir+science_files[i],overwrite=True)

del hdus
del hdrs

We also do this for the NGC6823 science frames. This time being in 2 sets.

In [31]:
science_files = ['a'+str(i).zfill(3)+'.fits' for i in np.arange(11)+153]

hdus = [fits.open(data_dir+file) for file in science_files]
hdrs = [hdu[0].header for hdu in hdus]

for i in range(np.size(hdus)):
    hdrs[i]['imagetyp'] = 'science'
    hdrs[i]['object']   = 'ngc6823'
    hdrs[i]['biassec']  = '[2141:2200,1:2048]' # FITS standard
    hdrs[i]['trimsec']  = '[54:2101,1:2048]'
    
    if ((i>=9) & (i<=11)):
        hdrs[i]['filter']   = 'U'
    elif ((i>=7) & (i<=8)):
        hdrs[i]['filter']   = 'B' 
    elif ((i>=4) & (i<=6)):
        hdrs[i]['filter']   = 'V'        
    elif ((i>=2) & (i<=3)):
        hdrs[i]['filter']   = 'R'        
    elif ((i>=0) & (i<=1)):
        hdrs[i]['filter']   = 'I'  
    
    hdus[i].writeto(reduced_dir+science_files[i],overwrite=True)

del hdus
del hdrs

We also do this for the PVSEP frames

In [33]:
science_files = ['a'+str(i).zfill(3)+'.fits' for i in np.arange(112, 130)]
print(science_files)
hdus = [fits.open(data_dir+file) for file in science_files]
hdrs = [hdu[0].header for hdu in hdus]

for i in range(np.size(hdus)):
    hdrs[i]['imagetyp'] = 'science'
    hdrs[i]['object']   = 'pvsep'
    hdrs[i]['biassec']  = '[2141:2200,1:2048]' # FITS standard
    hdrs[i]['trimsec']  = '[54:2101,1:2048]'
    
    if ((i>=9) & (i<=11)):
        hdrs[i]['filter']   = 'U'
    elif ((i>=7) & (i<=8)):
        hdrs[i]['filter']   = 'B' 
    elif ((i>=4) & (i<=6)):
        hdrs[i]['filter']   = 'V'        
    elif ((i>=2) & (i<=3)):
        hdrs[i]['filter']   = 'R'        
    elif ((i>=0) & (i<=1)):
        hdrs[i]['filter']   = 'I'  
    
    hdus[i].writeto(reduced_dir+science_files[i],overwrite=True)

del hdus
del hdrs

['a112.fits', 'a113.fits', 'a114.fits', 'a115.fits', 'a116.fits', 'a117.fits', 'a118.fits', 'a119.fits', 'a120.fits', 'a121.fits', 'a122.fits', 'a123.fits', 'a124.fits', 'a125.fits', 'a126.fits', 'a127.fits', 'a128.fits', 'a129.fits']


The M16 frames

In [34]:
science_files = ['a'+str(i).zfill(3)+'.fits' for i in np.arange(165, 182)]

hdus = [fits.open(data_dir+file) for file in science_files]
hdrs = [hdu[0].header for hdu in hdus]

for i in range(np.size(hdus)):
    hdrs[i]['imagetyp'] = 'science'
    hdrs[i]['object']   = 'm16'
    hdrs[i]['biassec']  = '[2141:2200,1:2048]' # FITS standard
    hdrs[i]['trimsec']  = '[54:2101,1:2048]'
    
    if ((i>=9) & (i<=11)):
        hdrs[i]['filter']   = 'U'
    elif ((i>=7) & (i<=8)):
        hdrs[i]['filter']   = 'B' 
    elif ((i>=4) & (i<=6)):
        hdrs[i]['filter']   = 'V'        
    elif ((i>=2) & (i<=3)):
        hdrs[i]['filter']   = 'R'        
    elif ((i>=0) & (i<=1)):
        hdrs[i]['filter']   = 'I'  
    
    hdus[i].writeto(reduced_dir+science_files[i],overwrite=True)

del hdus
del hdrs

The NGC7006 frames

In [35]:
science_files = ['a'+str(i).zfill(3)+'.fits' for i in np.arange(183,193)]

hdus = [fits.open(data_dir+file) for file in science_files]
hdrs = [hdu[0].header for hdu in hdus]

for i in range(np.size(hdus)):
    hdrs[i]['imagetyp'] = 'science'
    hdrs[i]['object']   = 'ngc7006'
    hdrs[i]['biassec']  = '[2141:2200,1:2048]' # FITS standard
    hdrs[i]['trimsec']  = '[54:2101,1:2048]'
    
    if ((i>=9) & (i<=11)):
        hdrs[i]['filter']   = 'U'
    elif ((i>=7) & (i<=8)):
        hdrs[i]['filter']   = 'B' 
    elif ((i>=4) & (i<=6)):
        hdrs[i]['filter']   = 'V'        
    elif ((i>=2) & (i<=3)):
        hdrs[i]['filter']   = 'R'        
    elif ((i>=0) & (i<=1)):
        hdrs[i]['filter']   = 'I'  
    
    hdus[i].writeto(reduced_dir+science_files[i],overwrite=True)

del hdus
del hdrs

And the M101 frames

In [36]:
science_files = ['a'+str(i).zfill(3)+'.fits' for i in np.arange(212,221)]

hdus = [fits.open(data_dir+file) for file in science_files]
hdrs = [hdu[0].header for hdu in hdus]

for i in range(np.size(hdus)):
    hdrs[i]['imagetyp'] = 'science'
    hdrs[i]['object']   = 'm101'
    hdrs[i]['biassec']  = '[2141:2200,1:2048]' # FITS standard
    hdrs[i]['trimsec']  = '[54:2101,1:2048]'
    
    if ((i>=9) & (i<=11)):
        hdrs[i]['filter']   = 'U'
    elif ((i>=7) & (i<=8)):
        hdrs[i]['filter']   = 'B' 
    elif ((i>=4) & (i<=6)):
        hdrs[i]['filter']   = 'V'        
    elif ((i>=2) & (i<=3)):
        hdrs[i]['filter']   = 'R'        
    elif ((i>=0) & (i<=1)):
        hdrs[i]['filter']   = 'I'  
    
    hdus[i].writeto(reduced_dir+science_files[i],overwrite=True)

del hdus
del hdrs

In [37]:
imgs = ImageFileCollection(reduced_dir,glob_include='[ad]???.fits',glob_exclude='*ot*.fits') # exclude master*, otz files in case of rerun

We set our model to be a third order 1D polynomial, subtract the overscan region, trim the image to be 2048x2048 for all frames and save each image into our reduced data directory.

In [40]:
model = Polynomial1D(3)

# loop over all frames and do subtraction and trim
for ccd, file_name in imgs.ccds(ccd_kwargs={'unit': 'adu'},return_fname=True):
    ccd = ccdp.subtract_overscan(ccd, fits_section=ccd.header['biassec'], median=False, model=model)  # subtract overscan, fits index
    ccd = ccdp.trim_image(ccd,fits_section=ccd.header['trimsec']) # trim images
    outfile = file_name.strip('.fits')+'ot.fits'
    ccd.write(reduced_dir+'/'+outfile, overwrite=True) # save the result

We now make a master bias from the overscan subtracted and trimmed frames with 3-sigma clipping

In [41]:
imgs = ccdp.ImageFileCollection(reduced_dir,glob_include='[ad]???ot.fits',glob_exclude='master*')
biases = imgs.files_filtered(imagetyp='bias', include_path=True)

master_bias = ccdp.combine(biases,
                             method='average',
                             sigma_clip=True, sigma_clip_low_thresh=3, sigma_clip_high_thresh=3,
                             sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std,
                             mem_limit=350e6, unit=u.adu
                            )

master_bias.meta['combined'] = True
master_bias.write(reduced_dir+'master_bias.fits',overwrite=True)

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:splitting each image into 11 chunks to limit memory usage to 350000000.0 bytes.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: splitting each image into 11 chunks to limit memory usage to 350000000.0 bytes. [ccdproc.combiner]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


...

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader inste

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
